In [208]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from scipy import optimize as op
import pandas as pd
import numpy as np

In [193]:
# model = Sequential()
# model.add(Dense(10, input_dim=4, activation='relu'))
# model.add(Dense(3, activation='softmax'))

In [194]:
# model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [195]:
# test = np.arange(70)/100.
# theta1 = test[:40] 
# theta2 = test[40:]

# theta1 = theta1.reshape((10, 4))
# theta2 = theta2.reshape((3, 10))

# dataset.sample(10)

In [196]:
dataset = pd.read_csv('iris.data', header=0)
dataset['class'] = dataset['class'].map({'Iris-setosa':0, 
                                         'Iris-versicolor':1, 
                                         'Iris-virginica':2}).astype(int)
X = dataset.as_matrix(['sepal length', 
                   'sepal width', 
                   'petal length', 
                   'petal width'])
y = dataset.as_matrix(['class'])
y = to_categorical(y)

In [197]:
def relu(x):
    return x * (x > 0)

def softmax(a):
    s = np.exp(a - np.max(a))
    if a.ndim <3: return s/s.sum(0) 

def f(theta):
    # Reshaping 1D input weights to 2 2D matrix.
    theta1 = theta[:40]
    theta2 = theta[40:]
    theta1 = theta1.reshape((10, 4))
    theta2 = theta2.reshape((3, 10))
    
    # Calculationg the loss via cross entropy.
    h = relu(theta1.dot(X.T))
    out = softmax(theta2.dot(h))
    loss = -np.log((y.T * out).sum(0))
    return loss.sum()

In [198]:
# z = relu(theta1.dot(X.T))
# out = softmax(theta2.dot(z))
# loss = np.log((y.T * out).sum(0))
# res = loss.sum()
# print res
print f(test)

339.2595348630145


In [207]:
#print np.exp(-100)
bounds = [(-2., 2.)]*70
op.differential_evolution(f, bounds, maxiter=50000)

     fun: 5.442223586104227
     jac: array([-0.16393615, -0.07652341, -0.06116148, -0.01696456,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.8680777 ,  0.37476182,  1.00422808,
        0.15105002, -0.26209666, -0.10197354, -0.23069342, -0.06678986,
        0.09525385,  0.16193207,  0.30298786, -0.0882606 ,  0.        ,
        0.        ,  0.        ,  0.        ,  4.29558078, -1.74929156,
        2.6896795 , -2.77067898,  0.44818513,  0.18478428,  0.34461785,
        0.0969699 ,  0.8226726 , -0.07463532,  0.66265509,  0.1571169 ,
       -0.02609317,  0.        ,  0.        , -0.05020411, -0.01844667,
       -0.03478497,  0.        ,  0.        , -0.07913394, -0.03343974,
        0.10035546,  0.        ,  0.        , -0.09064731,  0.64093548,
       -0.09540253,  0.        ,  0.1342241 ,  0.55628453, -0.19215722,
       -0.07426175,  0.        ,  0.        ,  0.14085177, -0.62248553,
        0.13018804,  0.   